In [162]:
my_apikey = 'f43934a981fc48f5926e5929d3ee0760' #designated the Cognitive Builder Faire
import json
import csv
import pandas as pd
import datetime
from datetime import timedelta, date
import time
import requests
import shutil
import os

In [180]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address = "1644 Platte St, Denver, CO"
list_of_parks = {'Confluence Park':'Confluence Park, Denver', 'Commons Park':'Commons Park, Denver','Skyline Park':'Skyline Park, Denver','Washington Park':'Washington Park, Denver'}

park_name = []
latitudes_list = []
longitudes_list = []

for address in list_of_parks:
    geolocator = Nominatim()
    location = geolocator.geocode(list_of_parks[address])
    latitude, longitude = location.latitude, location.longitude
    latitudes_list = latitudes_list + [latitude]
    longitudes_list = longitudes_list + [longitude]
    park_name = park_name + [address]
    #print latitude, longitude
   
parksDF = pd.DataFrame({'ParkName': park_name, 'latitude':latitudes_list,'longitude': longitudes_list})
parksDF.head()




,ParkName,latitude,longitude
0,Confluence Park,39.753757,-105.008485
1,Commons Park,39.757177,-105.005241
2,Washington Park,39.701081,-104.971034
3,Skyline Park,39.749463,-104.994227


In [199]:
import requests
import datetime

def get_weather_historical(latitude, longitude, startdate, enddate, measurement = "", units = "m"):
    ## Input: 
    ##   latitude
    ##   longitude
    ##   startdate (e.g., "20170608")
    ##   enddate (e.g., "20170609")
    ##   measurement (e.g., "temp") - leave blank ("") to retrieve all weather measurements
    ## Returns: 
    ##    a dataframe
    
    latitude_input = latitude
    longitude_input = longitude
    measurement_input = measurement
    
    possible_measurement = [None, u'', u'blunt_phrase',u'class', u'clds', u'day_ind', u'dewPt', u'expire_time_gmt', 
                            u'feels_like', u'gust', u'heat_index', u'icon_extd', u'key', u'max_temp', u'min_temp',
                            u'obs_id', u'obs_name', u'precip_hrly', u'precip_total', u'pressure', u'pressure_desc', 
                            u'pressure_tend', u'primary_swell_direction', u'primary_swell_height', u'primary_swell_period', 
                            u'primary_wave_height', u'primary_wave_period', u'qualifier', u'qualifier_svrty', u'rh',
                            u'secondary_swell_direction', u'secondary_swell_height', u'secondary_swell_period', u'snow_hrly',
                            u'temp', u'terse_phrase', u'uv_desc', u'uv_index', u'valid_time_gmt', u'vis', u'water_temp', u'wc', 
                            u'wdir', u'wdir_cardinal', u'wspd', u'wx_icon', u'wx_phrase']
    
    if measurement_input not in possible_measurement:
        print("ERROR: Measurement not valid. Please choose from:")
        print(possible_measurement[2:])
        return(None)
    
    timestamp = []
    measurement = []
    value = []
    df = pd.DataFrame()
    
    daterangelist = getDates(startdate,enddate,30)
    
    for daterange in daterangelist:
        print(daterange)
        url = "http://api.weather.com/v1/geocode/" + str(latitude_input) + "/" + str(longitude_input)+ \
            "/observations/historical.json?apiKey=" + my_apikey + \
            "&units=" + units + "&language=en-US" + \
            "&startDate="+str(daterange[0]) + \
            "&endDate="+str(daterange[1])
        
        r = requests.get(url).json()

        try: 
            if(r['errors']):
                return("[ERROR RETURNED FROM API REQUEST]: " + r['errors'][0]['error']['message'])
        except:
            pass
        
        for obs in range(1, len(r['observations'])):
            obstime = time.ctime(r['observations'][obs]['valid_time_gmt'])
            timestamp.extend([obstime] * len(r['observations'][0].keys()))
            measurement.extend(r['observations'][obs].keys())
            value.extend(r['observations'][obs].values())

        latitude = [r['metadata']['latitude']] * len(timestamp)
        longitude = [r['metadata']['longitude']] * len(timestamp)
        obs_name = [r['observations'][0]['obs_name']] * len(timestamp)

        newdf = pd.DataFrame(zip(obs_name, latitude, longitude, timestamp, measurement, value), 
                     columns= ['obs_name', 'latitude', 'longitude', 'timestamp_gmt', 'measurement', 'value'])
        
        df = df.append(newdf, ignore_index = True)
    
    
    #Fix date formatting
    df['timestamp_gmt'] = pd.to_datetime(df['timestamp_gmt'])
    df['date_gmt'] = df['timestamp_gmt'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    if measurement_input not in [u'', 'None']:
        #print(measurement)
        df = df[df.measurement == measurement_input]
        
    return(df.reset_index(drop=True))
def getDates(start_date, end_date, delta = 30):
    '''
    Returns a list of date intervals of delta number of days, that encompasses the entire duration between start_date and end_date
    start_date: starting date in format YYYYMMDD
    end_date: end date in format YYYYMMDD
    delta: set as number of days for the interval; the interval between start_date and end_date; default = 30
    '''
    from datetime import timedelta, date
    delta = delta -1
    #Convert dates to date format
    s = datetime.date(int(str(start_date)[:4]),int(str(start_date)[4:6]),int(str(start_date)[6:]))
    e = datetime.date(int(str(end_date)[:4]),int(str(end_date)[4:6]),int(str(end_date)[6:]))

    #Set delta
    if (int(delta) > 30 or int(delta) < 1):
        raise ValueError('Error: delta out of range. 1 <= delta <= 30')
    else:
        delta = timedelta(days=int(delta))

    days= []
    day_from = s
    while (day_from) < e:
        day_to = day_from + delta
        days.append((day_from.strftime("%Y%m%d"), day_to.strftime("%Y%m%d")))
        #print '...for range %s to %s ...' % (day_from.strftime("%Y%m%d"), day_to.strftime("%Y%m%d"))
        day_from += delta + timedelta(days=int(1))
    return days

In [198]:

dfHistorical = pd.DataFrame
for i in range(0, len(parksDF)):
    dftemp = get_weather_historical(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             startdate = "20170405",
                             enddate = "20170530",
                             measurement="temp")
    if i == 0:
        dfHistorical = dftemp
    else:
        dfHistorical = pd.concat([dfHistorical,dftemp])

    
dfHistorical.head()



,obs_name,latitude,longitude,timestamp_gmt,measurement,value,date_gmt
0,Aurora/Buckley Afb,39.75,-105.0,2017-04-05 01:58:00,temp,-2,2017-04-05
1,Aurora/Buckley Afb,39.75,-105.0,2017-04-05 02:58:00,temp,-2,2017-04-05
2,Aurora/Buckley Afb,39.75,-105.0,2017-04-05 03:58:00,temp,-3,2017-04-05
3,Aurora/Buckley Afb,39.75,-105.0,2017-04-05 04:58:00,temp,-3,2017-04-05
4,Aurora/Buckley Afb,39.75,-105.0,2017-04-05 05:58:00,temp,-4,2017-04-05


In [200]:
dfHistorical.tail()

,obs_name,latitude,longitude,timestamp_gmt,measurement,value,date_gmt
2358,Aurora/Buckley Afb,39.74,-104.99,2017-06-03 19:58:00,temp,19,2017-06-03
2359,Aurora/Buckley Afb,39.74,-104.99,2017-06-03 20:58:00,temp,16,2017-06-03
2360,Aurora/Buckley Afb,39.74,-104.99,2017-06-03 21:58:00,temp,13,2017-06-03
2361,Aurora/Buckley Afb,39.74,-104.99,2017-06-03 22:58:00,temp,13,2017-06-03
2362,Aurora/Buckley Afb,39.74,-104.99,2017-06-03 23:58:00,temp,13,2017-06-03


In [167]:
lat = 39.83
lon = -104.65
units = 'm'
url = "http://api.weather.com/v1/geocode/" + str(lat) + "/" + str(lon)+ \
    "/forecast/daily/3day.json?apiKey=" + my_apikey + \
    "&units=" + units
    

In [168]:
import requests
r = requests.get(url).json()
r

{u'forecasts': [{u'blurb': None,
   u'blurb_author': None,
   u'class': u'fod_long_range_daily',
   u'dow': u'Saturday',
   u'expire_time_gmt': 1497140990,
   u'fcst_valid': 1497099600,
   u'fcst_valid_local': u'2017-06-10T07:00:00-0600',
   u'lunar_phase': u'Waning Gibbous',
   u'lunar_phase_code': u'WNG',
   u'lunar_phase_day': 16,
   u'max_temp': None,
   u'min_temp': 16,
   u'moonrise': u'2017-06-10T21:22:41-0600',
   u'moonset': u'2017-06-10T06:35:44-0600',
   u'narrative': u'Windy with a few clouds. Low 16C.',
   u'night': {u'accumulation_phrase': u'',
    u'alt_daypart_name': u'Tonight',
    u'clds': 38,
    u'day_ind': u'N',
    u'daypart_name': u'Tonight',
    u'fcst_valid': 1497142800,
    u'fcst_valid_local': u'2017-06-10T19:00:00-0600',
    u'golf_category': u'',
    u'golf_index': None,
    u'hi': 33,
    u'icon_code': 24,
    u'icon_extd': 2910,
    u'long_daypart_name': u'Saturday night',
    u'narrative': u'Partly cloudy skies. Gusty winds during the evening. Low 16C. W

In [169]:
r.keys()

[u'forecasts', u'metadata']

In [170]:
len(r['forecasts'])

4

In [171]:
r['forecasts'][0]

{u'blurb': None,
 u'blurb_author': None,
 u'class': u'fod_long_range_daily',
 u'dow': u'Saturday',
 u'expire_time_gmt': 1497140990,
 u'fcst_valid': 1497099600,
 u'fcst_valid_local': u'2017-06-10T07:00:00-0600',
 u'lunar_phase': u'Waning Gibbous',
 u'lunar_phase_code': u'WNG',
 u'lunar_phase_day': 16,
 u'max_temp': None,
 u'min_temp': 16,
 u'moonrise': u'2017-06-10T21:22:41-0600',
 u'moonset': u'2017-06-10T06:35:44-0600',
 u'narrative': u'Windy with a few clouds. Low 16C.',
 u'night': {u'accumulation_phrase': u'',
  u'alt_daypart_name': u'Tonight',
  u'clds': 38,
  u'day_ind': u'N',
  u'daypart_name': u'Tonight',
  u'fcst_valid': 1497142800,
  u'fcst_valid_local': u'2017-06-10T19:00:00-0600',
  u'golf_category': u'',
  u'golf_index': None,
  u'hi': 33,
  u'icon_code': 24,
  u'icon_extd': 2910,
  u'long_daypart_name': u'Saturday night',
  u'narrative': u'Partly cloudy skies. Gusty winds during the evening. Low 16C. Winds SW at 30 to 50 km/h.',
  u'num': 1,
  u'phrase_12char': u'P Cldy/Wi

In [172]:
def get_weather_today(latitude, longitude, measurement = "", units = "m"):
    ## Input: 
    ##   latitude
    ##   longitude
    ##   startdate (e.g., "20170608")
    ##   enddate (e.g., "20170609")
    ##   measurement (e.g., "temp") - leave blank ("") to retrieve all weather measurements
    ## Returns: 
    ##    a dataframe
    
    latitude_input = latitude
    longitude_input = longitude
    measurement_input = measurement
    
    possible_measurement = [None, u'', u'blunt_phrase',u'class', u'clds', u'day_ind', u'dewPt', u'expire_time_gmt', 
                            u'feels_like', u'gust', u'heat_index', u'icon_extd', u'key', u'max_temp', u'min_temp',
                            u'obs_id', u'obs_name', u'precip_hrly', u'precip_total', u'pressure', u'pressure_desc', 
                            u'pressure_tend', u'primary_swell_direction', u'primary_swell_height', u'primary_swell_period', 
                            u'primary_wave_height', u'primary_wave_period', u'qualifier', u'qualifier_svrty', u'rh',
                            u'secondary_swell_direction', u'secondary_swell_height', u'secondary_swell_period', u'snow_hrly',
                            u'temp', u'terse_phrase', u'uv_desc', u'uv_index', u'valid_time_gmt', u'vis', u'water_temp', u'wc', 
                            u'wdir', u'wdir_cardinal', u'wspd', u'wx_icon', u'wx_phrase']
    
    if measurement_input not in possible_measurement:
        print("ERROR: Measurement not valid. Please choose from:")
        print(possible_measurement[2:])
        return(None)
    
    timestamp = []
    measurement = []
    value = []
    df = pd.DataFrame()
    

    
    url = "http://api.weather.com/v1/geocode/" + str(lat) + "/" + str(lon)+ \
    "/observations.json?apiKey=" + my_apikey + \
    "&units=" + units
    
    r = requests.get(url).json()
    try: 
        if(r['errors']):
            return("[ERROR RETURNED FROM API REQUEST]: " + r['errors'][0]['error']['message'])
    except:
        pass
        
 
    obstime = time.ctime(r['observation']['expire_time_gmt'])
    timestamp.extend([obstime] * len(r['observation'].keys()))
    measurement.extend(r['observation'].keys())
    value.extend(r['observation'].values())

    latitude = [r['metadata']['latitude']] * len(timestamp)
    longitude = [r['metadata']['longitude']] * len(timestamp)

    newdf = pd.DataFrame(zip(latitude, longitude, timestamp, measurement, value), 
                     columns= [ 'latitude', 'longitude', 'timestamp_gmt', 'measurement', 'value'])
        
    df = df.append(newdf, ignore_index = True)
    
    
    #Fix date formatting
    df['timestamp_gmt'] = pd.to_datetime(df['timestamp_gmt'])
    df['date_gmt'] = df['timestamp_gmt'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    if measurement_input not in [u'', 'None']:
        #print(measurement)
        df = df[df.measurement == measurement_input]
        
    return(df.reset_index(drop=True))

In [177]:
temperatures = []
uv_indexes = []
pressures = []
wspds = []
precip_totals = []
snow_hrlys = []


for i in range(0, len(parksDF)):
    #print parksDF.iloc[i]['latitude'], parksDF.iloc[i]['longitude']
    temp = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="temp")['value'][0]
    temperatures = temperatures + [temp]

    uv = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="uv_index")['value'][0]
    uv_indexes = uv_indexes + [uv]
    
    pressure = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="pressure")['value'][0]
    pressures = pressures + [pressure]
    
    wspd = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="wspd")['value'][0]
    wspds = wspds + [wspd]
    
    precip_total = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="precip_total")['value'][0]
    precip_totals = precip_totals + [precip_total]
    
    snow_hrly = get_weather_today(latitude = parksDF.iloc[i]['latitude'],
                             longitude = parksDF.iloc[i]['longitude'], 
                             measurement="snow_hrly")['value'][0]
    snow_hrlys = snow_hrlys + [snow_hrly]
    
  

parksDF['temp'] = temperatures
parksDF['uv_index'] = uv_indexes
parksDF['pressure'] = pressures
parksDF['wspd'] = wspds
parksDF['precip_total'] = precip_totals
parksDF['snow_qpf'] = snow_hrlys


parksDF.head()

,ParkName,latitude,longitude,temp,uv_index,pressure,wspd,precip_total,snow_qpf
0,Confluence Park,39.753757,-105.008485,33,1,821.05,35,None,None
1,Commons Park,39.757177,-105.005241,33,1,821.05,35,None,None
2,Washington Park,39.701081,-104.971034,33,1,821.05,35,None,None
3,Skyline Park,39.749463,-104.994227,33,1,821.05,35,None,None


In [146]:
lat = 39.7537572
lon = -105.00848512
units = 'm'
url = "http://api.weather.com/v1/geocode/" + str(lat) + "/" + str(lon)+ \
    "/observations.json?apiKey=" + my_apikey + \
    "&units=" + units